## Evolutionary algorithm to solve Travelling Salesman Problem

#### Representation: Permutation Representation
#### Recombination : Cut & Crossfill Crossover
#### Recombination probability : 100%
#### Mutation : Nearby swap
#### Mutation Probability : 50%
#### Parent Selection : Best 10 out of population
#### Population Size : 28
#### Survival selection : Replace worst
#### Number of offspring : 2
#### Initialization : Random
#### Termination condition : 100 fitness evaluation

In [98]:
import pandas as pd
import numpy as np
import random

In [99]:
#To find how many times fitness of our Genotype is checked. 
#This can be used as a metric to compare with other algorithm models
count_fitness = 0

In [100]:
#Distance matrix of the 7 cities
A = ( [ 0, 8, 2, 3, 7, 2, 5 ],
      [ 8, 0, 3, 5, 1, 8, 2 ],
      [ 2, 3, 0, 2, 2, 6, 8 ],
      [ 3, 5, 2, 0, 6, 2, 2 ],
      [ 7, 1, 2, 6, 0, 3, 5 ],
      [ 2, 8, 6, 2, 3, 0, 1 ],
      [ 5, 2, 8, 2, 5, 1, 0 ])

In [101]:
data = {'1st': [],
        '2nd': [],
        '3rd': [],
        '4th': [],
        '5th': [],
        '6th': [],
        '7th': []}
df = pd.DataFrame(data)

In [102]:
#Function to create n random parents as initial population
def random_n_parents(n):
    from numpy.random import default_rng
    rng = default_rng()
    for i in range(n):
        df.loc[i] = rng.choice(7, size=7, replace=False)
    return(df)

In [103]:
#10 random parents are being created as initial population
random_n_parents(10)
print("Random 10 initial parents are:\n",df)

Random 10 initial parents are:
    1st  2nd  3rd  4th  5th  6th  7th
0  3.0  2.0  0.0  1.0  6.0  5.0  4.0
1  0.0  4.0  6.0  5.0  2.0  1.0  3.0
2  5.0  6.0  1.0  3.0  2.0  0.0  4.0
3  1.0  4.0  3.0  2.0  0.0  6.0  5.0
4  0.0  2.0  4.0  1.0  6.0  3.0  5.0
5  3.0  4.0  2.0  6.0  0.0  1.0  5.0
6  6.0  1.0  3.0  0.0  4.0  5.0  2.0
7  3.0  4.0  0.0  1.0  6.0  5.0  2.0
8  6.0  4.0  2.0  1.0  5.0  0.0  3.0
9  2.0  6.0  0.0  5.0  4.0  1.0  3.0


Here first row 3-2-0-1-6-5-7 signifies the travelling salesman first travel to 3rd city then to 2nd city and so on. This is the Permutation representation (Genotype) of our Phenotype.

In [104]:
# Mutations are small changes happening to the Genotypes.
# Unlike Crossover's which moves fastly in search space Mutations makes small helping us to find local optimum's.
def mutation(C):
    x = random.randint(0,5)
    y = np.random.rand()
    if(y<0.5): # mutation probability 50%
        C[x],C[x+1] = C[x+1],C[x]
    return(C)
#Here we swap nearby 2 random values which gives a lesser impact for our Genotype but have greater relevance in EA's

In [105]:
# Cut and Crossfill Crossover
# By crossing 2 parents we create 2 new childs.

def crossover(P1,P2):
    k = random.randint(1,6)
    C1 = []*k
    C2 = []*k
    j = k + 1
    C1 = P1[:k] #Copying first part of parent 1 to child 1 (untill random cut "K")
    C2 = P2[:k] #Copying first part of parent 2 to child 2 (untill random cut "K")
    for i in range(7):
        z = 1
        for j in range(k):
            if(P2[i]==C1[j]):
                z = 0
                break
        if(z):
            C1 = np.append(C1,P2[i]) #Searching for remaining elements of C1 in P2 and copying 
    for i in range(7):
        z = 1
        for j in range(k):
            if(P1[i]==C2[j]):
                z = 0
                break
        if(z):
            C2 = np.append(C2,P1[i]) #Searching for remaining elements of C2 in P1 and copying 
    C1 = mutation(C1) # Childrens are mutated along with the crossover
    C2 = mutation(C2)
    return(C1,C2)
    

In [106]:
#Creating 9*2 = 18 new childs by crossing 9 parents
n=len(df)-1
for i in range(n):
    C1,C2 = crossover(df.iloc[i],df.iloc[i+1])
    df.loc[len(df)] = C1 #Adding new children to the population
    df.loc[len(df)] = C2
df.tail()

,1st,2nd,3rd,4th,5th,6th,7th
23,3.0,4.0,0.0,6.0,1.0,5.0,2.0
24,3.0,4.0,0.0,6.0,2.0,1.0,5.0
25,6.0,4.0,3.0,2.0,0.0,1.0,5.0
26,6.0,4.0,2.0,0.0,5.0,1.0,3.0
27,2.0,6.0,4.0,1.0,5.0,3.0,0.0


In [107]:
# Finding fitness of each Genotype
# Finding distance travelled by TS Man in each cases
# Minimizing this distance is our main objectice
def fitness(X):
    distance = 0
    distance = A[int(X[0])][int(X[1])] + A[int(X[1])][int(X[2])] + A[int(X[2])][int(X[3])]+ A[int(X[3])][int(X[4])] + A[int(X[4])][int(X[5])] + A[int(X[5])][int(X[6])] + A[int(X[6])][int(X[0])]
    return(distance)
#Distance to travel from first a city and move through all other cities and comeback

In [108]:
distance = []
for i in range(len(df)):
    distance = np.append(distance,fitness(df.loc[i]))
    count_fitness = count_fitness + 1
df['Fitness'] = distance
df.head()

,1st,2nd,3rd,4th,5th,6th,7th,Fitness
0,3.0,2.0,0.0,1.0,6.0,5.0,4.0,24.0
1,0.0,4.0,6.0,5.0,2.0,1.0,3.0,30.0
2,5.0,6.0,1.0,3.0,2.0,0.0,4.0,22.0
3,1.0,4.0,3.0,2.0,0.0,6.0,5.0,25.0
4,0.0,2.0,4.0,1.0,6.0,3.0,5.0,13.0


In [109]:
df = df.sort_values(by=['Fitness'], ascending=True)
df = df.drop_duplicates()
df.head()

,1st,2nd,3rd,4th,5th,6th,7th,Fitness
18,0.0,2.0,4.0,1.0,6.0,3.0,5.0,13.0
17,0.0,2.0,4.0,1.0,3.0,6.0,5.0,15.0
21,6.0,1.0,3.0,4.0,2.0,0.0,5.0,20.0
2,5.0,6.0,1.0,3.0,2.0,0.0,4.0,22.0
13,5.0,6.0,3.0,1.0,2.0,0.0,4.0,23.0


In [111]:
while(count_fitness < 100):
    df = df.head(10)
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    n=len(df)-1
    for i in range(n):
        C1,C2 = crossover(df.iloc[i],df.iloc[i+1])
        C1 = np.append(C1,fitness(C1))
        count_fitness = count_fitness + 1
        df.loc[len(df)] = C1
        C2 = np.append(C2,fitness(C2))
        count_fitness = count_fitness + 1
        df.loc[len(df)] = C2
    df = df.sort_values(by=['Fitness'], ascending=True)
    df = df.drop_duplicates()
    print(df.head())
    

    1st  2nd  3rd  4th  5th  6th  7th  Fitness
0   0.0  2.0  4.0  1.0  6.0  3.0  5.0     13.0
1   0.0  2.0  4.0  1.0  3.0  6.0  5.0     15.0
20  3.0  2.0  0.0  6.0  1.0  4.0  5.0     17.0
10  0.0  2.0  1.0  4.0  6.0  3.0  5.0     17.0
15  5.0  6.0  1.0  3.0  4.0  2.0  0.0     20.0
    1st  2nd  3rd  4th  5th  6th  7th  Fitness
0   0.0  2.0  4.0  1.0  6.0  3.0  5.0     13.0
1   0.0  2.0  4.0  1.0  3.0  6.0  5.0     15.0
18  5.0  6.0  3.0  1.0  4.0  2.0  0.0     15.0
2   3.0  2.0  0.0  6.0  1.0  4.0  5.0     17.0
3   0.0  2.0  1.0  4.0  6.0  3.0  5.0     17.0
    1st  2nd  3rd  4th  5th  6th  7th  Fitness
0   0.0  2.0  4.0  1.0  6.0  3.0  5.0     13.0
24  6.0  3.0  1.0  4.0  2.0  0.0  5.0     15.0
10  0.0  2.0  4.0  1.0  3.0  6.0  5.0     15.0
2   5.0  6.0  3.0  1.0  4.0  2.0  0.0     15.0
22  5.0  6.0  1.0  4.0  3.0  2.0  0.0     16.0
    1st  2nd  3rd  4th  5th  6th  7th  Fitness
0   0.0  2.0  4.0  1.0  6.0  3.0  5.0     13.0
20  0.0  3.0  2.0  4.0  1.0  6.0  5.0     13.0
3   5.0  6.0 

Here 13 is the optimum solution, which we got easily in the first iteration itself. This may not be the case every time. In every stage we chose 10 parents with optimum solution and these parents along with their childrens contribute next generation. After every generation we get more precise solutions. 

## Thanks and Regards
# JISHNU M
www.linkedin.com/in/jishnumanayathody